In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [2]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
caffe_root = '../../caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

In [6]:
import os
if os.path.isfile('./models/snapshots/itracker_iter_92000.caffemodel'):
    print ('Caffemodel found.')
else:
    print ('Caffemodel not found')

Caffemodel found.


In [44]:
caffe.set_mode_cpu()

model_def = './models/itracker_deploy.prototxt'
model_weights = './models/snapshots/itracker_iter_92000.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [45]:
# for reading .mat files
import scipy.io


In [69]:

# load the mean images
mu_face = scipy.io.loadmat('./models/mean_images/mean_face_224.mat')
mu_right = scipy.io.loadmat('./models/mean_images/mean_right_224.mat')
mu_left = scipy.io.loadmat('./models/mean_images/mean_left_224.mat')

# average over pixels to obtain the mean (RGB) pixel values
mu_face = mu_face['image_mean'].mean(0).mean(0)
mu_right = mu_right['image_mean'].mean(0).mean(0)  
mu_left = mu_left['image_mean'].mean(0).mean(0)  
print(mu_face.shape)

(3,)


In [70]:
# create transformers for the input data
transformer_image_face = caffe.io.Transformer({'image_face': net.blobs['image_face'].data.shape})
transformer_image_right = caffe.io.Transformer({'image_right': net.blobs['image_right'].data.shape})
transformer_image_left = caffe.io.Transformer({'image_left': net.blobs['image_left'].data.shape})
transformer_facegrid = caffe.io.Transformer({'facegrid': net.blobs['facegrid'].data.shape})



In [71]:
# move image channels to outermost dimension
transformer_image_face.set_transpose('image_face', (2,0,1))
transformer_image_right.set_transpose('image_right', (2,0,1))
transformer_image_left.set_transpose('image_left', (2,0,1))

In [72]:
# subtract the dataset-mean value in each channel
transformer_image_face.set_mean('image_face', mu_face) 
transformer_image_right.set_mean('image_right', mu_right)
transformer_image_left.set_mean('image_left', mu_left)